In [9]:
import psycopg2
import csv
from psycopg2.extras import RealDictCursor

In [5]:
%load_ext sql
%env DATABASE_URL=postgresql://localhost/vp_database1

The sql extension is already loaded. To reload it, use:
  %reload_ext sql
env: DATABASE_URL=postgresql://localhost/vp_database1


In [7]:
with psycopg2.connect(host='localhost', dbname='vp_database1') as conn:
    with conn.cursor(cursor_factory=RealDictCursor) as cursor:
        cursor.execute(
            """
            CREATE TABLE race (
                id SERIAL PRIMARY KEY,
                name TEXT
            )
            """
        )
        

In [14]:
with psycopg2.connect(host='localhost', dbname='vp_database1') as conn:
    with conn.cursor(cursor_factory=RealDictCursor) as cursor:

        with open('Race.csv') as csvfile:
          # tell python about the specific csv format
          myCSVReader = csv.DictReader(csvfile, delimiter=",", quotechar='"')

          # move row by row through the file as though it was a list.
          for row in myCSVReader:
                
                cursor.execute("SELECT id FROM race WHERE name = %(Race)s", 
                               row)

                # How many results?
                if (cursor.rowcount == 1):
                    # Only one result expected, use cursor.fetchone()
                    race_id = cursor.fetchone()['id']
                else:
                    cursor.execute("""
                                    INSERT INTO race(name)
                                    VALUES (
                                         %(Race)s -- name from CS
                                    ) 
                                    RETURNING id
                                    """,
                                    row)  
                    # store the id for the new venue
                    race_id = cursor.fetchone()['id']
                print(f"found {race_id} for {row['Race']}")

found 99 for White Alone
found 99 for White Alone
found 100 for Black or African American Alone
found 100 for Black or African American Alone
found 101 for American Indian & Alaska Native Alone
found 101 for American Indian & Alaska Native Alone
found 102 for Asian Alone
found 102 for Asian Alone
found 103 for Native Hawaiian & Other Pacific Islander Alone
found 103 for Native Hawaiian & Other Pacific Islander Alone
found 104 for Some Other Race Alone
found 104 for Some Other Race Alone
found 105 for Two or More Races
found 105 for Two or More Races
found 99 for White Alone
found 99 for White Alone
found 100 for Black or African American Alone
found 100 for Black or African American Alone
found 101 for American Indian & Alaska Native Alone
found 101 for American Indian & Alaska Native Alone
found 102 for Asian Alone
found 102 for Asian Alone
found 103 for Native Hawaiian & Other Pacific Islander Alone
found 103 for Native Hawaiian & Other Pacific Islander Alone
found 104 for Some Other

In [15]:
%%sql
SELECT * FROM race 

 * postgresql://localhost/vp_database1
7 rows affected.


id,name
99,White Alone
100,Black or African American Alone
101,American Indian & Alaska Native Alone
102,Asian Alone
103,Native Hawaiian & Other Pacific Islander Alone
104,Some Other Race Alone
105,Two or More Races


In [13]:
%%sql
TRUNCATE race

 * postgresql://localhost/vp_database1
Done.


[]

In [7]:
import csv


with psycopg2.connect(host='localhost', dbname='vp_database1') as conn:
    with conn.cursor(cursor_factory=RealDictCursor) as cursor:

        cursor.execute("SELECT * FROM BabyNames")
        
        with open('BabyNames.csv', 'w') as csvfile:
            # Declare csv fields in the order we want them
            column_names = [i[0] for i in cursor.description]
            myCsvWriter = csv.DictWriter(csvfile,
                                         fieldnames=column_names)

            myCsvWriter.writeheader()

            for row in cursor:           # <======== See extra indent
                myCsvWriter.writerow(row)

print("Done writing csv")

UndefinedTable: relation "babynames" does not exist
LINE 1: SELECT * FROM BabyNames
                      ^


In [8]:
with psycopg2.connect(host='localhost', dbname='vp_database1') as conn:
    with conn.cursor(cursor_factory=RealDictCursor) as cursor:

        with open('BabyNames.csv') as csvfile:
            myCSVReader = csv.DictReader(csvfile)

            sql = """
                  INSERT INTO venues(
                                 name,
                                 capacity
                              ) VALUES (
                                 %(name)s, 
                                 %(capacity)s
                              )
                  """

            for row in myCSVReader:
                # use row directly when csv headers match column names.
                print("Inserting: ")
                print(row)
                cursor.execute(sql, row)

Inserting: 
OrderedDict([('Year_of_Birth', '2011'), ('Gender', 'FEMALE'), ('Ethnicity', 'HISPANIC'), ('Childs_First_Name', 'GERALDINE'), ('Count', '13'), ('Rank', '75')])


KeyError: 'name'